# This is a PyBaMM test.
I want to create some PyBaMM data. Synthetic data.

In [1]:
import pybamm
import numpy as np
impoty pandas as pd
import matplotlib.pyplot as plt

SyntaxError: invalid syntax (3465959849.py, line 3)